In [5]:
import tvm
from tvm import relay
import onnx
from tvm.relay import testing
from tvm.contrib import graph_executor

In [6]:
# 1. Load the ONNX model
onnx_model = onnx.load("cifar10_onnx(1).onnx")

input_name = "input"
# Convert ONNX model to TVM relay format
mod, params = relay.frontend.from_onnx(onnx_model, shape={input_name: (1,32, 32,3)})

In [14]:
target = "llvm"  # Use "cuda" for GPU
dev = tvm.device(target, 0)  # Use tvm.gpu() for GPU

# Compile the model
with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mod, target=target, params=params)

In [15]:
import numpy as np
executor = graph_executor.GraphModule(lib["default"](dev))

# Prepare input data
input_data = np.random.rand(1, 3, 32, 32).astype(np.float32)

# Set input data
executor.set_input(input_name, tvm.nd.array(input_data, dev))  # Replace "input_name" with the correct name

# Run inference
executor.run()

In [16]:
output = executor.get_output(0).asnumpy()
print("Inference Output:", output)

Inference Output: [[4.3504439e-08 2.6197007e-05 2.7501959e-01 1.8509616e-03 1.2210029e-01
  1.2288886e-06 6.0099870e-01 1.6597917e-06 3.2078834e-08 1.4055591e-06]]


In [17]:
lib.export_library("tvm_model.so")

In [20]:
with open("model_graph.json","w") as f:
    f.write(lib.get_graph_json())